In [4]:
!pip install rapidfuzz

# Task 1 - Match user queries with resolved queries

In [10]:
# Step 0: Import libraries
import pandas as pd
from rapidfuzz import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load datasets
resolved = pd.read_csv("/kaggle/input/lab4textsearch/resolved_queries.csv")
new = pd.read_csv("/kaggle/input/lab4textsearch/new_queries.csv")

print("Resolved queries sample:\n", resolved.head(), "\n")
print("New queries sample:\n", new.head(), "\n")

# Use the correct column names
resolved_queries = resolved['Pre_Resolved_Query'].astype(str).tolist()
new_queries = new['Variation_Query'].astype(str).tolist()

# ===================================================================
# Step 2: Fuzzy Matching (RapidFuzz)
# ===================================================================
fuzzy_matches = []
for q in new_queries:
    match, score, idx = process.extractOne(q, resolved_queries, scorer=fuzz.token_sort_ratio)
    fuzzy_matches.append((q, match, score))

fuzzy_results = pd.DataFrame(fuzzy_matches, columns=["new_query", "best_match_fuzzy", "similarity_score"])
threshold = 80
fuzzy_results["is_match_fuzzy"] = fuzzy_results["similarity_score"] >= threshold

# ===================================================================
# Step 3: TF-IDF Matching
# ===================================================================
vectorizer = TfidfVectorizer()
tfidf_matrix_resolved = vectorizer.fit_transform(resolved_queries)
tfidf_matrix_new = vectorizer.transform(new_queries)

tfidf_matches = []
for i, q in enumerate(new_queries):
    cosine_similarities = cosine_similarity(tfidf_matrix_new[i], tfidf_matrix_resolved).flatten()
    best_idx = cosine_similarities.argmax()
    best_match = resolved_queries[best_idx]
    score = cosine_similarities[best_idx]
    tfidf_matches.append((q, best_match, score))

tfidf_results = pd.DataFrame(tfidf_matches, columns=["new_query", "best_match_tfidf", "similarity_score_tfidf"])

# ===================================================================
# Step 4: Evaluation
# ===================================================================
# Map resolved query → ID
resolved_dict = dict(zip(resolved['Pre_Resolved_Query'], resolved['Query_ID']))

# Map predictions
fuzzy_results['pred_Query_ID_fuzzy'] = fuzzy_results['best_match_fuzzy'].map(resolved_dict)
tfidf_results['pred_Query_ID_tfidf'] = tfidf_results['best_match_tfidf'].map(resolved_dict)

# Merge with ground truth
evaluation = new.merge(
    fuzzy_results[['new_query','pred_Query_ID_fuzzy']], 
    left_on='Variation_Query', right_on='new_query'
).merge(
    tfidf_results[['new_query','pred_Query_ID_tfidf']], 
    left_on='Variation_Query', right_on='new_query'
)

# Drop duplicate cols
evaluation = evaluation.drop(columns=['new_query_x','new_query_y'])

# Accuracy
acc_fuzzy = (evaluation['Matches_With_Query_ID'] == evaluation['pred_Query_ID_fuzzy']).mean()
acc_tfidf = (evaluation['Matches_With_Query_ID'] == evaluation['pred_Query_ID_tfidf']).mean()

print("Fuzzy Matching Accuracy:", acc_fuzzy)
print("TF-IDF Matching Accuracy:", acc_tfidf)

evaluation.head(10)

Resolved queries sample:
    Query_ID                    Pre_Resolved_Query
0         1     Unable to connect to the internet
1         2        Payment failed during checkout
2         3     App crashes when opening settings
3         4   Forgot password and unable to reset
4         5  Unable to upload files to the server 

New queries sample:
                              Variation_Query  Matches_With_Query_ID
0           Unabel to conect to the internet                      1
1                  Can’t connect to internet                      1
2                        Intenet not working                      1
3               Payment failed while chekout                      2
4  Payment did not go through during chckout                      2 

Fuzzy Matching Accuracy: 0.9
TF-IDF Matching Accuracy: 1.0


,Variation_Query,Matches_With_Query_ID,pred_Query_ID_fuzzy,pred_Query_ID_tfidf
0,Unabel to conect to the internet,1,1,1
1,Can’t connect to internet,1,1,1
2,Intenet not working,1,2,1
3,Payment failed while chekout,2,2,2
4,Payment did not go through during chckout,2,2,2
5,Payment issue at check out,2,2,2
6,Application crashes when opening setings,3,3,3
7,App crash when going to settings,3,3,3
8,Settings cause the app to chrash,3,1,3
9,Forgot passwrd and cant reset,4,4,4


- Performed Fuzzy Matching (RapidFuzz, token sort ratio)
- Performed TF-IDF + Cosine Similarity Matching
- Mapped matches to Query IDs
- Evaluated accuracy of both methods against ground truth (Matches_With_Query_ID)
- Printed accuracies + evaluation sample

# Task 2 - Match names

In [11]:
import pandas as pd
import re
import unicodedata
from rapidfuzz import fuzz, process

# ---------- Config ----------
BASE_PATH = "/kaggle/input/lab4textsearch/base_names.csv"
VAR_PATH  = "/kaggle/input/lab4textsearch/name_variations.csv"
OUTPUT_PATH = "/kaggle/working/matching_results.csv"
TOP_N = 3            # number of top candidates to keep (by token_set_ratio)
MATCH_THRESHOLD = 85 # ensemble score threshold to mark as confident match
# ----------------------------

# ---------- helpers ----------
def find_name_col(df):
    # heuristics to find name-like column; falls back to first column
    for c in df.columns:
        low = c.lower()
        if "name" in low or "full" in low:
            return c
    return df.columns[0]

def normalize_name(name):
    """lightweight normalization for names:
       - reorder 'Last, First' to 'First Last'
       - remove honorifics/titles, punctuation, parentheticals
       - collapse whitespace, lowercase
    """
    if pd.isna(name):
        return ""
    s = str(name)
    s = unicodedata.normalize("NFKD", s)
    # remove content in parentheses
    s = re.sub(r"\(.*?\)", " ", s)
    # handle trailing comma format "Last, First Middle"
    if "," in s:
        parts = [p.strip() for p in s.split(",") if p.strip()]
        if len(parts) >= 2:
            s = " ".join(parts[1:] + [parts[0]])
    # remove dots, hyphens -> spaces (so J.D. -> J D)
    s = s.replace(".", " ").replace("-", " ")
    s = s.strip()
    s = s.lower()
    # remove common honorifics/titles
    s = re.sub(r"\b(?:mr|mrs|ms|miss|dr|prof|sir|madam|mx)\b\.?", " ", s)
    # keep only letters and spaces (removes numbers, punctuation)
    s = re.sub(r"[^a-z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---------- load data ----------
base_df = pd.read_csv(BASE_PATH)
var_df  = pd.read_csv(VAR_PATH)

base_col = find_name_col(base_df)
var_col  = find_name_col(var_df)

base_orig = base_df[base_col].astype(str).tolist()
var_orig  = var_df[var_col].astype(str).tolist()

# ---------- normalized lists (keeps alignment) ----------
base_norms = [normalize_name(x) for x in base_orig]
var_norms  = [normalize_name(x) for x in var_orig]

# ---------- matching ----------
rows = []
scorers = {
    "token_set": fuzz.token_set_ratio,
    "token_sort": fuzz.token_sort_ratio,
    "partial": fuzz.partial_ratio,
    "ratio": fuzz.ratio
}

# Pre-build choices for rapidfuzz (processor=None since we've pre-normalized)
choices = base_norms  # same index mapping as base_orig

for i, (orig_var, norm_var) in enumerate(zip(var_orig, var_norms)):
    # top-N by token_set (often good for name variations)
    top_n = process.extract(norm_var, choices, scorer=fuzz.token_set_ratio, processor=None, limit=TOP_N)
    top_n_out = []
    for match_norm, score, idx in top_n:
        top_n_out.append({
            "matched_base_name": base_orig[idx],
            "matched_base_norm": match_norm,
            "score": score,
            "base_index": idx
        })

    # best per scorer
    per_scorer = {}
    for sname, scorer in scorers.items():
        best = process.extractOne(norm_var, choices, scorer=scorer, processor=None)
        if best is None:
            per_scorer[sname] = {"base": None, "score": 0, "idx": None}
        else:
            match_norm, score, idx = best
            per_scorer[sname] = {"base": base_orig[idx], "score": score, "idx": idx}

    # ensemble decision: choose candidate with highest single-score across scorers
    # (simple, robust choice: pick candidate with max score among all scorers)
    ensemble_candidates = []
    for sname, info in per_scorer.items():
        if info["base"] is not None:
            ensemble_candidates.append((info["base"], info["score"], sname, info["idx"]))
    if ensemble_candidates:
        ensemble_best = max(ensemble_candidates, key=lambda x: x[1])  # (base, score, scorer, idx)
        ensemble_base, ensemble_score, ensemble_scorer, ensemble_idx = ensemble_best
    else:
        ensemble_base, ensemble_score, ensemble_scorer, ensemble_idx = (None, 0, None, None)

    # boolean flag if confident match
    is_confident = ensemble_score >= MATCH_THRESHOLD

    rows.append({
        "Variation_Name": orig_var,
        "Variation_Name_norm": norm_var,
        # top-N results (as compact string)
        "Top{}_Candidates_token_set".format(TOP_N): " | ".join(
            [f"{r['matched_base_name']} ({r['score']:.0f})" for r in top_n_out]
        ) if top_n_out else "",
        # per-scorer bests + scores
        **{f"Best_{sname}_match": per_scorer[sname]["base"] for sname in scorers},
        **{f"Score_{sname}": per_scorer[sname]["score"] for sname in scorers},
        # ensemble
        "Ensemble_best_match": ensemble_base,
        "Ensemble_best_score": ensemble_score,
        "Ensemble_best_scorer": ensemble_scorer,
        "Is_confident_match": is_confident
    })

# ---------- output DataFrame & save ----------
out_df = pd.DataFrame(rows)
out_df.index.name = "Variation_index"

# Add mapping back to any ID columns in base/vars if present
# (optional: if base_df has Query_ID or similar, you can map that here)
# Example: if base_df has 'ID' column, you could map ensemble match -> its ID by searching base_orig

out_df.to_csv(OUTPUT_PATH, index=True)
print(f"Finished matching. Results saved to: {OUTPUT_PATH}")
print("\nSample rows:")
print(out_df.head(12).to_string(index=False))

# ---------- quick summary ----------
num_confident = out_df["Is_confident_match"].sum()
print(f"\nTotal variations: {len(out_df)}, Confident matches (>= {MATCH_THRESHOLD}): {num_confident}")


Finished matching. Results saved to: /kaggle/working/matching_results.csv

Sample rows:
Variation_Name Variation_Name_norm Top3_Candidates_token_set Best_token_set_match Best_token_sort_match Best_partial_match Best_ratio_match  Score_token_set  Score_token_sort  Score_partial  Score_ratio Ensemble_best_match  Ensemble_best_score Ensemble_best_scorer  Is_confident_match
   Thomas King         thomas king     1 (0) | 2 (0) | 3 (0)                    1                     1                  1                1              0.0               0.0            0.0          0.0                   1                  0.0            token_set               False
   Thomas King         thomas king     1 (0) | 2 (0) | 3 (0)                    1                     1                  1                1              0.0               0.0            0.0          0.0                   1                  0.0            token_set               False
  Maria Garcia        maria garcia     1 (0) | 2 (0) | 3 

- auto-detects the name column in each CSV,
- normalizes names (handles Last, First forms, punctuation, titles, extra spaces),
- runs multiple fuzzy scorers (token_set_ratio, token_sort_ratio, partial_ratio, ratio),
- returns top-3 candidates (by token_set_ratio) and best match per scorer,
- picks an ensemble best (highest score across scorers),
- flags matches that meet a configurable threshold,
- saves results to matching_results.csv.